### [연속 펄스 부분 수열의 합](https://school.programmers.co.kr/learn/courses/30/lessons/161988?language=python3)

- 부분 수열의 최대값은 누적합 간의 차이의 최대값으로 측정이 가능하며
- 아래와 같은 펄스가 적용될 경우 최대값과 최소값 간의 차이(크기)가 그 값이 됨

In [ ]:
def solution(sequence):
    prefix_sums = [0]
    curr_sum = 0
    
    for i, val in enumerate(sequence):
        if i % 2 == 0:
            curr_sum += val
        else:
            curr_sum -= val
        prefix_sums.append(curr_sum)

    return max(prefix_sums) - min(prefix_sums)
